In [21]:
import zarr
import json

In [3]:
crop50_path='./jrc_hela-3/recon-1/labels/groundtruth/crop50/labels/all'

In [4]:
zarr_array = zarr.open(raw_path, mode='r') # mode='r'
crop50 = zarr.open(crop50_path, mode='r') # mode='r'

In [5]:
zarr_array.info

Type,zarr.core.Array
Data type,uint8
Shape,"(12000, 1000, 12400)"
Chunk shape,"(128, 128, 128)"
Order,C
Read-only,True
Compressor,Zstd(level=6)
Store type,zarr.storage.DirectoryStore
No. bytes,148800000000 (138.6G)
No. bytes stored,330
Storage ratio,450909090.9


In [6]:
crop50.shape

(512, 512, 512)

In [20]:
# from_coords = (7184, 318, 6255)  # Start coordinates (example)
# to_coords = (7684, 568, 6755)   # End coordinates (example)
from_coords = (6608, 82, 9273)  # Start coordinates (example)
to_coords = (6863, 337, 9528)   # End coordinates (example)

Offset information

In [22]:
# Specify the path to your JSON file
json_file_path = './jrc_hela-3/dataset/attrs'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Extract and print the 'offset' values for each crop under 'subvolumes'
for crop, details in data['subvolumes'].items():
    print(f"{crop} offset: {details['offset']}")


Crop111 offset: [28736, 1272, 25020]
Crop27 offset: [22800.0, 3200.0, 14360.0]
Crop33 offset: [13800.0, 920.0, 22600.0]
Crop34 offset: [26680.0, 2600.0, 23600.0]
Crop50 offset: [37092.0, 328.0, 26432.0]
Crop60 offset: [7752.0, 2040.0, 22864.0]
Crop61 offset: [43896.0, 2260.0, 16748.0]
Crop62 offset: [6416.0, 2436.0, 15804.0]
Crop63 offset: [25600.0, 2592.0, 46136.0]
Crop64 offset: [43560.0, 2688.0, 22716.0]
Crop65 offset: [4184.0, 2492.0, 19760.0]
Crop85 offset: [20440.0, 1040.0, 23200.0]
Crop86 offset: [26480.0, 1080.0, 21720.0]
Crop87 offset: [20120.0, 1280.0, 17480.0]


In [14]:
# Specify the parent directory path
parent_path = './jrc_hela-3/recon-1/labels/groundtruth'

In [20]:
# Open the Zarr group at the parent path
group = zarr.open_group(parent_path, mode='r')

# List of directories to exclude
exclude_dirs = ['0003', '0004']

# Iterate over each sub-group under groundtruth
for crop_key in group.group_keys():
    if crop_key in exclude_dirs:
        continue  # Skip excluded directories

    crop_group = group[crop_key]  # Access the cropX group
    print(f"Contents of {crop_key}:")
    
    # Print the structure of each crop group
    print(crop_group.tree())
    
    # Check if 'labels' exists in the group
    if 'labels' in crop_group:
        labels_group = crop_group['labels']
        if 'all' in labels_group:
            all_group = labels_group['all']
            # Check if 'all_group' is a group or an array
            if isinstance(all_group, zarr.Group):
                for array_key in all_group.array_keys():
                    array = all_group[array_key]
                    print(f"Array {array_key}: shape {array.shape}, dtype {array.dtype}")
            elif isinstance(all_group, zarr.Array):
                print(f"Array 'all': shape {all_group.shape}, dtype {all_group.dtype}")
    
    print("\n")


Contents of crop111:
crop111
 └── labels
     ├── all (1000, 500, 1000) uint64
     └── ribosomes (1000, 500, 1000) uint64
Array 'all': shape (1000, 500, 1000), dtype uint64


Contents of crop27:
crop27
 └── labels
     └── all (400, 400, 400) uint64
Array 'all': shape (400, 400, 400), dtype uint64


Contents of crop33:
crop33
 └── labels
     └── all (400, 400, 400) uint64
Array 'all': shape (400, 400, 400), dtype uint64


Contents of crop34:
crop34
 └── labels
     └── all (400, 400, 400) uint64
Array 'all': shape (400, 400, 400), dtype uint64


Contents of crop50:
crop50
 └── labels
     └── all (512, 512, 512) uint64
Array 'all': shape (512, 512, 512), dtype uint64


Contents of crop51:
crop51
 └── labels
     └── all (512, 512, 512) uint64
Array 'all': shape (512, 512, 512), dtype uint64


Contents of crop60:
crop60
 └── labels
     └── all (400, 400, 400) uint64
Array 'all': shape (400, 400, 400), dtype uint64


Contents of crop61:
crop61
 └── labels
     └── all (400, 400, 400) 

In [32]:
import json
import zarr

# Step 1: Read Zarr contents and gather crop shapes
parent_path = './jrc_hela-3/recon-1/labels/groundtruth/'
group = zarr.open_group(parent_path, mode='r')

# Dictionary to store crop contents
crop_info_dict = {}

# List of directories to exclude
exclude_dirs = ['0003', '0004']

# Iterate over each cropX folder under groundtruth and store the contents
for crop_key in group.group_keys():
    if crop_key in exclude_dirs:
        continue  # Skip excluded directories

    crop_group = group[crop_key]  # Access the cropX group

    if 'labels' in crop_group:
        labels_group = crop_group['labels']
        if 'all' in labels_group:
            all_group = labels_group['all']
            if isinstance(all_group, zarr.Array):
                # Extract crop shape from the array
                crop_shape = all_group.shape
                # Initialize crop info with shape
                crop_info_dict[crop_key] = {'crop_shape': crop_shape}

# Step 2: Load the JSON file and extract offsets
json_file_path = './jrc_hela-3/dataset/attrs'

with open(json_file_path, 'r') as file:
    data = json.load(file)

# Step 3: Match the crop offsets with the shapes and create the desired dictionary
for crop, details in data['subvolumes'].items():
    crop_name = crop.lower()  # Convert to lowercase to match the key in the dictionary
    offset = details['offset']
    
    if crop_name in crop_info_dict:
        # Add offset to the existing dictionary
        crop_info_dict[crop_name]['offset'] = offset

# Step 4: Output the final dictionary
crop_info_dict


{'crop111': {'crop_shape': (1000, 500, 1000), 'offset': [28736, 1272, 25020]},
 'crop27': {'crop_shape': (400, 400, 400),
  'offset': [22800.0, 3200.0, 14360.0]},
 'crop33': {'crop_shape': (400, 400, 400),
  'offset': [13800.0, 920.0, 22600.0]},
 'crop34': {'crop_shape': (400, 400, 400),
  'offset': [26680.0, 2600.0, 23600.0]},
 'crop50': {'crop_shape': (512, 512, 512),
  'offset': [37092.0, 328.0, 26432.0]},
 'crop51': {'crop_shape': (512, 512, 512)},
 'crop60': {'crop_shape': (400, 400, 400),
  'offset': [7752.0, 2040.0, 22864.0]},
 'crop61': {'crop_shape': (400, 400, 400),
  'offset': [43896.0, 2260.0, 16748.0]},
 'crop62': {'crop_shape': (400, 400, 400),
  'offset': [6416.0, 2436.0, 15804.0]},
 'crop63': {'crop_shape': (400, 400, 400),
  'offset': [25600.0, 2592.0, 46136.0]},
 'crop64': {'crop_shape': (400, 400, 400),
  'offset': [43560.0, 2688.0, 22716.0]},
 'crop65': {'crop_shape': (400, 400, 400),
  'offset': [4184.0, 2492.0, 19760.0]},
 'crop85': {'crop_shape': (400, 400, 400),

In [36]:
N = 4 # raw file offset denom
M = 2

In [41]:
# Iterate over the dictionary and modify values directly
for crop, details in crop_info_dict.items():
    if 'crop_shape' in details:
        # Calculate crop_shape / M for each dimension and update directly
        crop_info_dict[crop]['crop_shape'] = tuple(dim / M for dim in details['crop_shape'])
    
    if 'offset' in details:
        # Calculate offset / N for each value and update directly
        crop_info_dict[crop]['offset'] = [val / N for val in details['offset']]

# Output the modified dictionary
crop_info_dict

{'crop111': {'crop_shape': (250.0, 125.0, 250.0),
  'offset': [1796.0, 79.5, 1563.75],
  'crop_shape_divided': (500.0, 250.0, 500.0),
  'offset_divided': [7184.0, 318.0, 6255.0]},
 'crop27': {'crop_shape': (100.0, 100.0, 100.0),
  'offset': [1425.0, 200.0, 897.5],
  'crop_shape_divided': (200.0, 200.0, 200.0),
  'offset_divided': [5700.0, 800.0, 3590.0]},
 'crop33': {'crop_shape': (100.0, 100.0, 100.0),
  'offset': [862.5, 57.5, 1412.5],
  'crop_shape_divided': (200.0, 200.0, 200.0),
  'offset_divided': [3450.0, 230.0, 5650.0]},
 'crop34': {'crop_shape': (100.0, 100.0, 100.0),
  'offset': [1667.5, 162.5, 1475.0],
  'crop_shape_divided': (200.0, 200.0, 200.0),
  'offset_divided': [6670.0, 650.0, 5900.0]},
 'crop50': {'crop_shape': (128.0, 128.0, 128.0),
  'offset': [2318.25, 20.5, 1652.0],
  'crop_shape_divided': (256.0, 256.0, 256.0),
  'offset_divided': [9273.0, 82.0, 6608.0]},
 'crop51': {'crop_shape': (128.0, 128.0, 128.0),
  'crop_shape_divided': (256.0, 256.0, 256.0)},
 'crop60': 

In [34]:
raw_path = './jrc_hela-3/recon-1/em/fibsem-uint8/s0'
zarr_array = zarr.open(raw_path, mode='r')

# Print the shape of the Zarr array to verify
print(f"Zarr array shape: {zarr_array.shape}")

Zarr array shape: (12000, 1000, 12400)


In [49]:
# Iterate over each item in the dictionary 
for crop_key, crop_info in crop_info_dict.items():
    print(crop_key)
    
    # Check if 'offset' and 'crop_shape' keys exist in the dictionary entry
    if 'offset_divided' in crop_info and 'crop_shape_divided' in crop_info:
        offset = crop_info['offset_divided']
        crop_shape = crop_info['crop_shape_divided']
    
        # Define the coordinates for cropping (ensure they are in z, y, x order and within the shape bounds)
        from_coords = [int(offset[0]), int(offset[1]), int(offset[2])]  # Convert offsets to integers
        to_coords = [int(from_coords[0] + crop_shape[0]),
                    int(from_coords[1] + crop_shape[1]),
                    int(from_coords[2] + crop_shape[2])]
        # Check if the coordinates are within the array's bounds
        if (0 <= from_coords[0] < zarr_array.shape[0] and 0 <= to_coords[0] <= zarr_array.shape[0] and
            0 <= from_coords[1] < zarr_array.shape[1] and 0 <= to_coords[1] <= zarr_array.shape[1] and
            0 <= from_coords[2] < zarr_array.shape[2] and 0 <= to_coords[2] <= zarr_array.shape[2]):
            print("from_coords", from_coords[2])
            print("from_coords", from_coords[1])
            print("from_coords", from_coords[0])
            print("to_coords", to_coords[2])
            print("to_coords", to_coords[1])
            print("to_coords", to_coords[0])
            
            # Extract the crop using slicing
            cropped_chunk = zarr_array[
                from_coords[2]:to_coords[2],
                from_coords[0]:to_coords[0],   # x dimension
                from_coords[1]:to_coords[1],  # y dimension
            ]
            # print("cropped_chunk", cropped_chunk)

            # Save the cropped chunk to a new Zarr array
            output_path = f'./jrc_hela-3/raw_crops/{crop_key}.zarr'
            zarr.save(output_path, cropped_chunk)

            print(f'{crop_key} - Cropped chunk shape: {cropped_chunk.shape} saved to {output_path}')
        else:
            print(f"Error: Specified coordinates for {crop_key} are out of bounds.")
    else:
        print(f"Warning: {crop_key} does not have 'offset' or 'crop_shape' information.")


crop111
from_coords 6255
from_coords 318
from_coords 7184
to_coords 6755
to_coords 568
to_coords 7684
crop111 - Cropped chunk shape: (500, 0, 250) saved to ./jrc_hela-3/raw_crops/crop111.zarr
crop27
from_coords 3590
from_coords 800
from_coords 5700
to_coords 3790
to_coords 1000
to_coords 5900
crop27 - Cropped chunk shape: (200, 0, 200) saved to ./jrc_hela-3/raw_crops/crop27.zarr
crop33
from_coords 5650
from_coords 230
from_coords 3450
to_coords 5850
to_coords 430
to_coords 3650
crop33 - Cropped chunk shape: (200, 0, 200) saved to ./jrc_hela-3/raw_crops/crop33.zarr
crop34
from_coords 5900
from_coords 650
from_coords 6670
to_coords 6100
to_coords 850
to_coords 6870
crop34 - Cropped chunk shape: (200, 0, 200) saved to ./jrc_hela-3/raw_crops/crop34.zarr
crop50
from_coords 6608
from_coords 82
from_coords 9273
to_coords 6864
to_coords 338
to_coords 9529
crop50 - Cropped chunk shape: (256, 0, 256) saved to ./jrc_hela-3/raw_crops/crop50.zarr
crop51
crop60
from_coords 5716
from_coords 510
from_